In [1]:
import tensorflow.compat.v2 as tf
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pathlib
import random
import glob
import os
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
import time
keras = tf.keras
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])
    image = tf.cast(image, tf.float32)
    image = image/255.0
    image = 2*image-1
    return image

In [3]:
train_dir = r'C:\Users\24306\Desktop\datasets\kfold\2\train'
test_dir  = r'C:\Users\24306\Desktop\datasets\kfold\2\test'
checkpoint_path = r'C:\Users\24306\Desktop\EfficientNetV2L\2\EfficientNetV2L.ckpt'

# 处理训练数据

In [4]:
#打乱顺序
train_root = pathlib.Path(train_dir)
train_image_paths = list(train_root.glob('*/*'))
train_image_paths = [str(path) for path in train_image_paths]
random.shuffle(train_image_paths)
#处理训练标签
train_image_labels = [pathlib.Path(path).parent.name for path in train_image_paths]
train_labels = [[label.split('_')[0]] for label in train_image_labels]
new_train_labels=[]
for n in train_labels:
    new_train_labels.append(float(n[0]));  
train_labels=new_train_labels
print(train_labels)
#处理训练图像
train_path_ds = tf.data.Dataset.from_tensor_slices(train_image_paths)
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_image_ds = train_path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
train_label_ds = tf.data.Dataset.from_tensor_slices(train_labels)
train_image_label_ds = tf.data.Dataset.zip((train_image_ds, train_label_ds))
# image_count = len(all_image_paths)

# test_count = int(image_count*0.2)
train_count = len(train_image_paths)
# train_data = image_label_ds.skip(test_count)
# test_data = image_label_ds.take(test_count)

BATCH_SIZE = 32
train_data = train_image_label_ds.shuffle(buffer_size=train_count).repeat(-1)
train_data = train_data.batch(BATCH_SIZE)
train_data = train_data.prefetch(buffer_size=AUTOTUNE)

#test_data = test_data.batch(BATCH_SIZE)
print(train_data)



[100.0, 16.0, 4.0, 4.0, 16.0, 86.0, 7.0, 66.0, 84.0, 45.0, 13.0, 80.0, 100.0, 10.0, 84.0, 45.0, 45.0, 45.0, 20.0, 96.0, 16.0, 86.0, 23.0, 52.0, 100.0, 13.0, 13.0, 86.0, 95.0, 80.0, 96.0, 93.0, 37.0, 95.0, 93.0, 7.0, 30.0, 96.0, 47.0, 100.0, 80.0, 35.0, 13.0, 23.0, 84.0, 37.0, 37.0, 45.0, 10.0, 66.0, 96.0, 66.0, 84.0, 37.0, 47.0, 13.0, 16.0, 86.0, 37.0, 13.0, 80.0, 32.0, 93.0, 40.0, 16.0, 86.0, 40.0, 7.0, 80.0, 37.0, 32.0, 96.0, 86.0, 47.0, 66.0, 47.0, 37.0, 66.0, 13.0, 16.0, 96.0, 100.0, 39.0, 84.0, 80.0, 96.0, 96.0, 80.0, 96.0, 23.0, 7.0, 20.0, 84.0, 55.0, 84.0, 47.0, 37.0, 7.0, 7.0, 55.0, 86.0, 13.0, 55.0, 37.0, 80.0, 95.0, 7.0, 80.0, 96.0, 7.0, 37.0, 30.0, 96.0, 16.0, 4.0, 47.0, 95.0, 86.0, 45.0, 32.0, 16.0, 37.0, 100.0, 30.0, 13.0, 13.0, 32.0, 47.0, 95.0, 7.0, 32.0, 30.0, 13.0, 13.0, 86.0, 86.0, 23.0, 35.0, 100.0, 40.0, 84.0, 95.0, 37.0, 86.0, 93.0, 95.0, 45.0, 52.0, 37.0, 37.0, 35.0, 13.0, 7.0, 96.0, 23.0, 55.0, 95.0, 66.0, 13.0, 45.0, 4.0, 40.0, 52.0, 66.0, 45.0, 95.0, 66.0, 55.0

# 处理验证数据

In [5]:
#打乱顺序
test_root = pathlib.Path(test_dir)
test_image_paths = list(test_root.glob('*/*'))
test_image_paths = [str(path) for path in test_image_paths]
#random.shuffle(train_image_paths)
#处理训练标签
test_image_labels = [pathlib.Path(path).parent.name for path in test_image_paths]
test_labels = [[label.split('_')[0]] for label in test_image_labels]
new_test_labels=[]
for n in test_labels:
    new_test_labels.append(float(n[0]));  
test_labels=new_test_labels
print(test_labels)
#处理训练图像
test_path_ds = tf.data.Dataset.from_tensor_slices(test_image_paths)
AUTOTUNE = tf.data.experimental.AUTOTUNE
test_image_ds = test_path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
test_label_ds = tf.data.Dataset.from_tensor_slices(test_labels)
test_image_label_ds = tf.data.Dataset.zip((test_image_ds, test_label_ds))
# image_count = len(all_image_paths)

# test_count = int(image_count*0.2)
test_count = len(test_image_paths)
# train_data = image_label_ds.skip(test_count)
# test_data = image_label_ds.take(test_count)

BATCH_SIZE = 32
test_data = test_image_label_ds.shuffle(buffer_size=test_count).repeat(-1)
test_data = test_data.batch(BATCH_SIZE)
test_data = test_data.prefetch(buffer_size=AUTOTUNE)

#test_data = test_data.batch(BATCH_SIZE)
print(test_data)

[10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.

In [6]:
#要改
EfficientNetV2L = tf.keras.applications.efficientnet_v2.EfficientNetV2L(weights='imagenet',include_top=False,input_shape=(224, 224, 3))
#要改
EfficientNetV2L.trainable = True
inputs = tf.keras.Input(shape=(224, 224, 3))
#要改
x = EfficientNetV2L(inputs)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
out_weight = tf.keras.layers.Dense(1,name='out_weight')(x)

model = tf.keras.Model(inputs=inputs,
                       outputs=[out_weight])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 efficientnetv2-l (Functiona  (None, 7, 7, 1280)       117746848 
 l)                                                              
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 out_weight (Dense)          (None, 1)                 1281      
                                                                 
Total params: 117,748,129
Trainable params: 117,235,553
Non-trainable params: 512,576
_________________________________________________________________


In [7]:
print ('start time:',time.asctime( time.localtime(time.time()) ))
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, decay=0.01),loss='mse')

cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,save_best_only=True,monitor='val_loss')
history = model.fit(train_data,
                    steps_per_epoch=train_count//BATCH_SIZE,
                    epochs=300,
                    validation_data=test_data,
                    validation_steps=test_count//BATCH_SIZE,
                   callbacks=[cp_callback])
#要改
np.savetxt('EfficientNetV2L_loss_2.txt',history.history['loss'])
#要改
np.savetxt('EfficientNetV2L_valloss_2.txt',history.history['val_loss'])
print ('end time:',time.asctime( time.localtime(time.time()) ))

start time: Sat May  6 09:04:00 2023
Epoch 1/300
591/591 [==============================] - ETA: 0s - loss: 870.3676

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\EfficientNetV2L\2\EfficientNetV2L.ckpt\assets


INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\EfficientNetV2L\2\EfficientNetV2L.ckpt\assets


591/591 [==============================] - 564s 882ms/step - loss: 870.3676 - val_loss: 146.4721
Epoch 2/300
591/591 [==============================] - ETA: 0s - loss: 153.5427

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\EfficientNetV2L\2\EfficientNetV2L.ckpt\assets


...

591/591 [==============================] - 1169s 2s/step - loss: 0.4364 - val_loss: 0.1358
Epoch 85/300
591/591 [==============================] - 926s 2s/step - loss: 0.3701 - val_loss: 0.2109
Epoch 86/300
591/591 [==============================] - 936s 2s/step - loss: 0.3216 - val_loss: 0.4928
Epoch 87/300
591/591 [==============================] - 934s 2s/step - loss: 0.5340 - val_loss: 0.2591
Epoch 88/300
591/591 [==============================] - 934s 2s/step - loss: 0.4151 - val_loss: 0.8080
Epoch 89/300
591/591 [==============================] - 930s 2s/step - loss: 0.6589 - val_loss: 0.4128
Epoch 90/300
591/591 [==============================] - 923s 2s/step - loss: 0.4019 - val_loss: 3.1676
Epoch 91/300
591/591 [==============================] - 922s 2s/step - loss: 0.3193 - val_loss: 0.3109
Epoch 92/300
591/591 [==============================] - 925s 2s/step - loss: 0.3256 - val_loss: 1.2993
Epoch 93/300
591/591 [==============================] - 932s 2s/step - loss: 0.2726 -

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\EfficientNetV2L\2\EfficientNetV2L.ckpt\assets


INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\EfficientNetV2L\2\EfficientNetV2L.ckpt\assets


591/591 [==============================] - 1219s 2s/step - loss: 0.2938 - val_loss: 0.1070
Epoch 127/300
591/591 [==============================] - 989s 2s/step - loss: 0.2047 - val_loss: 0.9918
Epoch 128/300
591/591 [==============================] - 968s 2s/step - loss: 0.2422 - val_loss: 0.1947
Epoch 129/300
591/591 [==============================] - 973s 2s/step - loss: 0.2446 - val_loss: 0.4231
Epoch 130/300
591/591 [==============================] - 971s 2s/step - loss: 0.3043 - val_loss: 0.2081
Epoch 131/300
591/591 [==============================] - 978s 2s/step - loss: 0.3151 - val_loss: 0.2670
Epoch 132/300
591/591 [==============================] - 990s 2s/step - loss: 0.4803 - val_loss: 1.4615
Epoch 133/300
591/591 [==============================] - 992s 2s/step - loss: 0.2651 - val_loss: 0.6791
Epoch 134/300
591/591 [==============================] - 991s 2s/step - loss: 0.2027 - val_loss: 0.1425
Epoch 135/300
591/591 [==============================] - 977s 2s/step - loss:

591/591 [==============================] - 1087s 2s/step - loss: 0.2347 - val_loss: 2.9606
Epoch 283/300
591/591 [==============================] - 1170s 2s/step - loss: 0.2680 - val_loss: 0.1922
Epoch 284/300
591/591 [==============================] - 1206s 2s/step - loss: 0.1328 - val_loss: 3.5200
Epoch 285/300
591/591 [==============================] - 1218s 2s/step - loss: 0.1225 - val_loss: 2.0796
Epoch 286/300
591/591 [==============================] - 1202s 2s/step - loss: 0.1218 - val_loss: 1.0459
Epoch 287/300
591/591 [==============================] - 1147s 2s/step - loss: 0.1442 - val_loss: 0.1567
Epoch 288/300
591/591 [==============================] - 1129s 2s/step - loss: 0.1711 - val_loss: 0.2349
Epoch 289/300
591/591 [==============================] - 1141s 2s/step - loss: 0.1608 - val_loss: 0.1315
Epoch 290/300
591/591 [==============================] - 1126s 2s/step - loss: 0.1348 - val_loss: 2.8801
Epoch 291/300
591/591 [==============================] - 994s 2s/step